### 串行集成 Boosting
  
###### 思想
先从初始训练集训练出一个基学习器，再根据基学习器的表现对训练样本分布进行调整，使得先前基学习器做错的训练样本后续受到更多关注，然后基于调整后的样本分布来训练下一个基学习器；如此重复进行，直至基学习器数目达到事先指定的值T，最终将这T个基学习器进行加权结合。   
    
  
#### 代表算法AdaBoost
给定数据集：$X=\{(x_1,y_1),(x_2,y_2),\cdots,(x_m,y_m)\quad\} \quad y_i \in\{-1,1\}$  
  
真实函数为$f$;集成器为一系列基学习器的加权和：$H(x)=\sum_{i=1}^T \alpha_i h_i(x) \quad$训练样本服从的分布$D$  

  
损失函数：$e^{-f(x)H(x)}\qquad$可以看出H(x)正确分类:$e^{-1}\qquad$H(x)错误分类：$e^{1}\qquad$  
  
   
$\qquad\qquad l_{exp}(H|D)\;=\;E_{x_{\tilde{}}\;D}[e^{-f(x)H(x)}]$   
  
目标是使这个损失函数最小；错误分类样本后续的概率变大，对损失函数的影响越大；所以我们最小化损失函数的时候对这个错分样本关注越多。    
<br>
  
$\frac{\partial \; l_{exp}\;(H|D)}{\partial \;H(x)}\;\;=\;\;E_{x_{\tilde{}}\;D}[-f(x)e^{-f(x)H(x)}]\;\;=\;\;-e^{-H(x)}p\left(f(x)=1|x\right)+e^{H(x)}p\left(f(x)=-1|x\right)$   
  
<br>  

偏导为0得：$\quad H(x)=\frac{1}{2} ln\left(\frac{p\left(f(x)=1|x\right)}{p\left(f(x)=-1|x\right)} \right)$  
  
    
由于是二分类:$sign(\;H(x)\;)=sign\left(\frac{1}{2} ln\frac{p\left(f(x)=1|x\right)}{p\left(f(x)=-1|x\right)} \right)=\begin{cases} 1, & p\left(f(x)=1|x\right)>p\left(f(x)=-1|x\right) \\ -1, &p\left(f(x)=1|x\right)<p\left(f(x)=-1|x\right) \end{cases} \qquad =\;\;\underset{y\in\{-1,1\}}{arg\;max}\;p\left(f(x)=y|x\right)$  
   
<br>   
  
这里验证了使用指数损失函数$e^{-f(x)H(x)}\quad$得到的结果。和最大后验概率的结果相同，所以可以使用这个指数损失函数。  
  
  
<br> 

* 基分类器的权重$\alpha$：  
  
求出一个新的基分类器$h_t$后，如何确定这个基分类器在集成器中的权重$\alpha_t$：  
  
集成器：$H=\alpha_1h_1+\alpha_2h_2+\cdots+\alpha_th_t$ 那么加上新的分类器后$\alpha_th_t$ 后，损失函数的最小值应该更小了(前面t-1的$\alpha_t$已经确定)(指示函数（true）=1  ；（false）=0  )。  
  
  
$l_{exp}(H|D_t)\;=\;E_{x_{\tilde{}}\;D_t}[e^{-f(x)(\alpha_1h_1+\alpha_2h_2+\cdots+\alpha_th_t)}]=E_{x_{\tilde{}}\;D_t}[e^{-f(x)\alpha_1h_1}e^{-f(x)\alpha_2h_2}\cdots e^{-f(x)\alpha_th_t)}]$  
  
$\qquad \;=E_{x_{\tilde{}}\;D_t}[e^{-[](f(x)=H_t(x))}+e^{[](f(x)\neq H_t(x))}]$ (每一个x两项中只有一项成立；最后n个x的项中提取$e^{-\alpha}$则对应的概率相加为(1-错误率);$e^{\alpha}$对应的概率相加为错误率$\epsilon$) 
  
  
$\qquad \;=Ce^{-\alpha_t}*P_{x_{\tilde{}}\;D_{\;t}}f(x)=h_t(x))+Ce^{\alpha_t}*P_{x_{\tilde{}}\;D_{\;t}}f(x)\neq h_t(x))$  
  
 $\qquad \;=Ce^{-\alpha_t}*P_{x_{\tilde{}}\;D_{\;t}}f(x)=h_t(x))+Ce^{\alpha_t}*P_{x_{\tilde{}}\;D_{\;t}}f(x)\neq h_t(x))$  
   
 $\qquad \;=Ce^{-\alpha_t}*(1-\epsilon_t)+Ce^{\alpha_t}*\epsilon_t$  
     
  <br>
   
 $\qquad  \frac{\partial \; l_{exp}\;(H|D)}{\partial \;\alpha_t}=-Ce^{-\alpha_t}*(1-\epsilon_t)+Ce^{\alpha_t}*\epsilon_t=0 $  
   
 $\qquad \alpha_t=\frac{1}{2}ln \frac{1-\epsilon_t}{\epsilon_t}$   
   
     
 <br>   
 <br>  
 
  
* 下次训练的分布$D_t$:  

训练出一个基分类器h_t，和权重$\alpha_t$之后，要根据这次的分类情况进行数据集分布的调整。  

  
如训练好$h_{t-1}$分类器了集成器为：$H_{t-1}$ ,则要调整$h_{t}$分类器所需要的分布$D_t$;理想情况下在分布$D_t$下训练的 $h_{t}$分类器能够纠正：$H_{t-1}$的所有错误。  
  
也就是最小化:   
$l_{exp}\;(H_{t-1}+h_t|D_t)=E_{x_{\tilde{}}\;D_t}[e^{-f(x）（H_{t-1}+h_t）}\;\;]$   
  
$l_{exp}\;(H_{t-1}+h_t|D_t)=E_{x_{\tilde{}}\;D_t}[e^{-f(x）H_{t-1}}\;\;e^{-f(x）h_t}\;\;]$   
  
泰勒展开： $l_{exp}\;(H_{t-1}+h_t|D_t)=E_{x_{\tilde{}}\;D_t}[e^{-f(x）H_{t-1}}\;\;\left( 1-f(x)h_t(x)+\frac{f^2(x)h_t^2(x)}{2}\right)\;\;]$   
  
    
    
  $l_{exp}\;(H_{t-1}+h_t|D_t)=E_{x_{\tilde{}}\;D_t}[e^{-f(x）H_{t-1}}\;\;\left( 1-f(x)h_t(x)+\frac{1}{2}\right)\;\;]$   
    
  在分布$D_t$下训练的 $h_{t}$理想的分类器 $h_t=\underset{h}{arg\;min}\;\;l_{exp}\;(H_{t-1}+h_t|D_t)$  
    
    
   $h_t=\underset{h}{arg\;min}\;\;E_{x_{\tilde{}}\;D_t}[e^{-f(x）H_{t-1}}\;\;\left( 1-f(x)h_t(x)+\frac{1}{2}\right)]$  
     
       
  $h_t=\underset{h}{arg\;max}\;\;E_{x_{\tilde{}}\;D_t}[(-f(x)h_t(x))*e^{-f(x）H_{t-1}}]$   
    
 $h_t=\underset{h}{arg\;max}\;\;E_{x_{\tilde{}}\;D_t}[(-f(x)h_t(x))*\frac{e^{-f(x）H_{t-1}}}{E_{x_{\tilde{}}\;D_t}[e^{-f(x）H_{t-1}}]}]$   
   
   